# Roadtripper

A simple notebook for roadtrip map creation.

Goal: 
A method that allows the selection of multiple points and 

In [13]:
# Imports
from geopy.geocoders import Nominatim
from pathlib import Path
import requests
import folium
import pandas as pd
from PIL import Image
import io
import json
import numpy as np
from uuid import uuid1

In [15]:
f"{uuid1()}"

'52abbd31-9763-11ee-8a6d-4889e7c7b329'

In [2]:
if Path("api.key").exists():
  with open("api.key", "r") as f:
    API_KEY = f.readlines()[0]
else:
  print("Key file 'api.key' missing!")

API_KEY

'd742964759msh0a9f0f8192af060p190631jsn5fde4f16baee'

In [36]:
# Lat-Long from str
lat_long_by_name = {}

def get_lat_long_from_address(address: str | tuple) -> (float, float):
   if type(address) == tuple:
      return address

   if not address in lat_long_by_name:
      locator = Nominatim(user_agent=f"{uuid1()}")
      location = locator.geocode(address)
      if location == None:
         print(f"Address '{address}' not found!")
         exit(-1)
      lat_long_by_name[address] = location.latitude, location.longitude
      with open("locations.json", "w") as f:
         json.dump(lat_long_by_name, f)
      
   return lat_long_by_name[address]

In [4]:
# Direction Response
def get_directions_response(lat1: float, long1: float, lat2: float, long2: float, mode: str='drive'):
   url = "https://route-and-directions.p.rapidapi.com/v1/routing"
   key = API_KEY
   host = "route-and-directions.p.rapidapi.com"
   headers = {"X-RapidAPI-Key": key, "X-RapidAPI-Host": host}
   querystring = {"waypoints":f"{str(lat1)},{str(long1)}|{str(lat2)},{str(long2)}","mode":mode}
   # print(url, headers)
   response = requests.request("GET", url, headers=headers, params=querystring)
   return response

In [5]:
def dist(a:tuple[float], b:tuple[float]):
  assert len(a) == len(b) == 2
  return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

In [6]:
def create_map(responses, lat_lons, force_points_every_n:int=0):
   m = folium.Map()
   df = pd.DataFrame()   # add markers for the places we visit
   for point in lat_lons:
      folium.Marker(point).add_to(m)   # loop over the responses and plot the lines of the route
   for response in responses:
      mls = response.json()['features'][0]['geometry']['coordinates']
      points = []
      skipped_points = 0
      for i, point in enumerate(mls[0]):
         if 0 < i < len(mls[0]) - 1 and force_points_every_n > 0 and force_points_every_n > skipped_points:
            skipped_points += 1
            continue
         skipped_points = 0
         points.append((point[1], point[0]))
      
      # add the lines
      folium.PolyLine(points, weight=5, opacity=1).add_to(m)      
      temp = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
      df = pd.concat([df, temp])   # create optimal zoom
   sw = df[['Lat', 'Lon']].min().values.tolist()
   sw = [sw[0]-0.0005, sw[1]-0.0005]
   ne = df[['Lat', 'Lon']].max().values.tolist()
   ne = [ne[0]+0.0005, ne[1]+0.0005]
   m.fit_bounds([sw, ne])
   return m

In [7]:
def map_creator(addresses: list[str], name:str, force_points_every_n:int=0):
  lat_lons = [get_lat_long_from_address(addr) for addr in addresses]
  responses = []
  for n in range(len(lat_lons)-1):
    lat1, lon1, lat2, lon2 = lat_lons[n][0], lat_lons[n][1], lat_lons[n+1][0], lat_lons[n+1][1]
    response = get_directions_response(lat1, lon1, lat2, lon2)
    responses.append(response)

  m = create_map(responses, [], force_points_every_n)
  m.save(f"res/{name}.html")
  img_data = m._to_png(5)
  img = Image.open(io.BytesIO(img_data))
  img.save(f'res/{name}.png')
  return m

## Roadtrip Australia

In [12]:
addresses = [
  "Ropes Crossing, New South Wales 2760, Australia",
  "Government Rd, Nelson Bay NSW 2315, Australia",
  "34 Kinka Rd, Seal Rocks NSW 2423, Australia",
  "4451 The Lakes Way, Pacific Palms NSW 2428, Australia",
  "8 Jonson St, Byron Bay NSW 2481, Australia",
  "69 Hyde St, Bellingen NSW 2454, Australia",
  "70 Glen Innes Rd, Armidale NSW 2350, Australia",
  "2814 Putty Rd, Milbrodale NSW 2330, Australia",
  "Ropes Crossing, New South Wales 2760, Australia"
]

map_creator(addresses, f"nsw", 100)

GeocoderInsufficientPrivileges: Non-successful status code 403

In [37]:
addresses = [
  "1 Creswells Row, Hobart TAS 7000, Australia",
  "LOT 1 Garden Point Rd, Port Arthur TAS 7182, Australia",
  "6 George St, Nubeena TAS 7184, Australia",
  "75 Freycinet Dr, Coles Bay TAS 7215, Australia",
  "Ansons Bay Rd, Ansons Bay TAS 7264, Australia",
  "1965 Main Rd, Lilydale TAS 7268, Australia",
  "270 York St, Launceston TAS 7250, Australia",
  "330 Mayberry Rd, Mayberry TAS 7304, Australia",
  "Dove Lake Rd, Cradle Mountain TAS 7306, Australia",
  "65 Batchelor St, Queenstown TAS 7467, Australia",
  "Lake Dobson, Australia",
  "153 Kingfish Beach Rd, Southport TAS 7109, Australia",
  "42 Mary St, Cygnet TAS 7112, Australia",
  "1750 Lighthouse Rd, South Bruny TAS 7150, Australia",
  "696 Adventure Bay Rd, Adventure Bay TAS 7150, Australia",
  "Dunn Pl, Hobart TAS 7000, Australia"
]

map_creator(addresses, "tas", 100)

1 Creswells Row, Hobart TAS 7000, Australia
LOT 1 Garden Point Rd, Port Arthur TAS 7182, Australia
6 George St, Nubeena TAS 7184, Australia
75 Freycinet Dr, Coles Bay TAS 7215, Australia
Ansons Bay Rd, Ansons Bay TAS 7264, Australia
1965 Main Rd, Lilydale TAS 7268, Australia
270 York St, Launceston TAS 7250, Australia


KeyboardInterrupt: 

In [35]:
a = (-43.732009, 170.093779)
type(a) == tuple[float, float]

False

In [43]:
addresses = [
  "197 Montgomerie Road, Māngere, Auckland 2022, New Zealand",
  "1446 Colville Road, Coromandel 3584, New Zealand",
  "715 Hot Water Beach Road, Hot Water Beach 3591, New Zealand",
  "26 Paku Drive, Tairua 3508, New Zealand",
  "1029 Hinemoa Street, Rotorua 3010, New Zealand",
  "Timona Park Road, Feilding 4702, New Zealand",
  "95 Hutt Park Road, Moera, Lower Hutt 5010, New Zealand",
  "3 Blind Creek Road, Marlborough, Tuamarina 7273, New Zealand", 
  "47 Montgomery Square, Nelson 7010, New Zealand",
  "1147 Takaka Hill Highway, Takaka Hill 7073, New Zealand",
  "14 Harvey Road, Mārahau 7197, New Zealand",
  "244 Palmerston Street, Westport 7825, New Zealand",
  "60 Herbert Street, Greymouth 7805, New Zealand",
  "20 Dunmore Street, Wānaka 9305, New Zealand",
  "4 Cemetery Road, Queenstown 9300, New Zealand",
  "23 Waiau Street, Manapouri 9679, New Zealand",
  "79 Milford Sound Highway, Milford Sound 9679, New Zealand",
  "103 Malvern Street, Woodhaugh, Dunedin 9010, New Zealand",
  "4587 Tekapo-Twizel Road, Pukaki 7999, New Zealand",
  (-43.732009, 170.093779),
  "227 Marine Parade, New Brighton, Christchurch 8061, New Zealand",
  "Kaikōura Peninsula Walkway, Kaikōura Peninsula, Kaikōura 7300, New Zealand",
  "517 Evans Bay Parade, Hataitai, Wellington 6021, New Zealand",
  "25 South Road, Manaia 4612, New Zealand",
  (-39.328644, 173.803522),
  "1d Beach Street, Fitzroy, New Plymouth 4312, New Zealand",
  "124 Regan Street, Stratford 4332, New Zealand",
  (-39.236010, 175.555039),
  (-38.693797, 175.688752),
  "1495 Pukuatua Street, Rotorua 3010, New Zealand",
  (-38.035290, 176.247341),
  "58 Ascot Road, Māngere, Auckland 2022, New Zealand"
]

map_creator(addresses, "nz", 100)